In [8]:
from langchain_community.document_loaders import TextLoader #makrdown file also
from langchain_community.document_loaders.csv_loader import CSVLoader
# from langchain_community.document_loaders.csv_loader import JSONLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")

In [9]:
loader = WebBaseLoader(web_path='https://jalammar.github.io/illustrated-bert/')

In [10]:
documents = loader.load()

In [11]:
documents

[Document(metadata={'source': 'https://jalammar.github.io/illustrated-bert/', 'title': 'The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning) – Jay Alammar – Visualizing machine learning one concept at a time.', 'description': 'Discussions:\nHacker News (98 points, 19 comments), Reddit r/MachineLearning (164 points, 20 comments)\n\n\nTranslations: Chinese (Simplified), French 1, French 2, Japanese, Korean, Persian, Russian, Spanish\n\n2021 Update: I created this brief and highly accessible video intro to BERT\n\n\n\n\n\nThe year 2018 has been an inflection point for machine learning models handling text (or more accurately, Natural Language Processing or NLP for short). Our conceptual understanding of how best to represent words and sentences in a way that best captures underlying meanings and relationships is rapidly evolving. Moreover, the NLP community has been putting forward incredibly powerful components that you can freely download and use in your own models an

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter()
splited_docs = text_splitter.split_documents(documents=documents)

In [14]:
splited_docs

[Document(metadata={'source': 'https://jalammar.github.io/illustrated-bert/', 'title': 'The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning) – Jay Alammar – Visualizing machine learning one concept at a time.', 'description': 'Discussions:\nHacker News (98 points, 19 comments), Reddit r/MachineLearning (164 points, 20 comments)\n\n\nTranslations: Chinese (Simplified), French 1, French 2, Japanese, Korean, Persian, Russian, Spanish\n\n2021 Update: I created this brief and highly accessible video intro to BERT\n\n\n\n\n\nThe year 2018 has been an inflection point for machine learning models handling text (or more accurately, Natural Language Processing or NLP for short). Our conceptual understanding of how best to represent words and sentences in a way that best captures underlying meanings and relationships is rapidly evolving. Moreover, the NLP community has been putting forward incredibly powerful components that you can freely download and use in your own models an

In [16]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3.1",
)

In [17]:
from langchain_chroma import Chroma
db = Chroma.from_documents(splited_docs, embedding=embeddings) 

In [21]:
db.similarity_search(query="what is BERT")

[Document(metadata={'description': 'Discussions:\nHacker News (98 points, 19 comments), Reddit r/MachineLearning (164 points, 20 comments)\n\n\nTranslations: Chinese (Simplified), French 1, French 2, Japanese, Korean, Persian, Russian, Spanish\n\n2021 Update: I created this brief and highly accessible video intro to BERT\n\n\n\n\n\nThe year 2018 has been an inflection point for machine learning models handling text (or more accurately, Natural Language Processing or NLP for short). Our conceptual understanding of how best to represent words and sentences in a way that best captures underlying meanings and relationships is rapidly evolving. Moreover, the NLP community has been putting forward incredibly powerful components that you can freely download and use in your own models and pipelines (It’s been referred to as NLP’s ImageNet moment, referencing how years ago similar developments accelerated the development of machine learning in Computer Vision tasks).\n\n\n  \n\n\n\n', 'language

In [20]:
retriver = db.as_retriever()
retriver

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002ACFE830940>)

In [22]:
quiry = "What is transformers in BERT"

In [23]:
retriver.invoke(quiry)

[Document(metadata={'description': 'Discussions:\nHacker News (98 points, 19 comments), Reddit r/MachineLearning (164 points, 20 comments)\n\n\nTranslations: Chinese (Simplified), French 1, French 2, Japanese, Korean, Persian, Russian, Spanish\n\n2021 Update: I created this brief and highly accessible video intro to BERT\n\n\n\n\n\nThe year 2018 has been an inflection point for machine learning models handling text (or more accurately, Natural Language Processing or NLP for short). Our conceptual understanding of how best to represent words and sentences in a way that best captures underlying meanings and relationships is rapidly evolving. Moreover, the NLP community has been putting forward incredibly powerful components that you can freely download and use in your own models and pipelines (It’s been referred to as NLP’s ImageNet moment, referencing how years ago similar developments accelerated the development of machine learning in Computer Vision tasks).\n\n\n  \n\n\n\n', 'language

In [24]:
from langchain_core.prompts import ChatPromptTemplate
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriver,question_answer_chain)

In [30]:
question_answer_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002AC82757EB0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002AC8275DA80>, d

In [31]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002ACFE830940>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input

In [33]:
res = rag_chain.invoke({"input":"How BERT was built..?"})

In [34]:
res["answer"]

'BERT is built upon a Transformer encoder stack, which is a type of neural network architecture. It uses a masked language model concept, where it masks 15% of the input words and asks the model to predict the missing word. BERT also includes a two-sentence classification task, where the model learns to predict if two sentences are likely to follow each other. \n'